In [2]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm
from scipy import stats


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
os.chdir('/home2/s439906/project/CASP16/monomer/')

In [3]:
raw_data_path = './monomer_data_final/raw_data/'
files = [file for file in os.listdir(raw_data_path) if file.endswith('.csv') and file.startswith('T1')]
files = sorted(files)
# files

In [4]:
GDT_HA_percentile_dict = {}
for file in files:
    # read data
    data = pd.read_csv(raw_data_path + file, index_col=0)
    # get GDT_HA column
    gdt_ha = data['GDT_HA']
    # get the 95% percentile
    percentile_95 = gdt_ha.quantile(0.95)
    EU = file.split('.')[0]
    if EU not in GDT_HA_percentile_dict:
        GDT_HA_percentile_dict[EU] = percentile_95
    else:
        print('Error: EU already in dict')
        break

In [ ]:
species_information={
    "T1201-D1": "high eukaryotes",
    "T1206-D1": "viral",
    "T1207-D1": "viral",
    "T1208s1-D1": "prokaryotes",
    "T1208s2-D1": "prokaryotes",
    "T1210-D1": "high eukaryotes",
    "T1212-D1": "transposon",
    "T1218-D1": "prokaryotes",
    "T1218-D2": "prokaryotes",
    "T1218-D3": "prokaryotes",
    "T1220s1-D1": "viral",
    "T1226-D1": "viral",
    "T1227s1-D1": "prokaryotes",
    "T1228-D1": "prokaryotes",
    "T1228-D2": "prokaryotes",
    "T1228-D3": "prokaryotes",
    "T1228-D4": "prokaryotes",
    "T1230s1-D1": "viral",
    "T1231-D1": "viral",
    "T1234-D1": "viral",
    "T1235-D1": "viral",
    "T1237-D1": "prokaryotes",
    "T1239-D1": "prokaryotes",
    "T1239-D2": "prokaryotes",
    "T1239-D3": "prokaryotes",
    "T1239-D4": "prokaryotes",
    "T1240-D1": "prokaryotes",
    "T1240-D2": "prokaryotes",
    "T1243-D1": "prokaryotes",
    "T1244s1-D1": "low eukaryotes",
    "T1245s1-D1": "singleton",
    "T1245s2-D1": "singleton",
    "T1246-D1": "prokaryotes",
    "T1249-D1": "viral",
    "T1249v1-D1": "viral", 
    # there is a problem in the nomenclature of the target, it should be T1249v1-D1, but T1249-D1 is used in other places. we will keep both for now
    "T1257-D1": "viral",
    "T1259-D1": "prokaryotes",
    "T1266-D1": "high eukaryotes",
    "T1267s1-D1": "prokaryotes",
    "T1267s1-D2": "prokaryotes",
    "T1267s2-D1": "prokaryotes",
    "T1269-D1": "high eukaryotes",
    "T1269-D2": "high eukaryotes",
    "T1269-D3": "high eukaryotes",
    "T1270-D1": "prokaryotes",
    "T1270-D2": "prokaryotes",
    "T1271s1-D1": "low eukaryotes",
    "T1271s2-D1": "low eukaryotes",
    "T1271s3-D1": "low eukaryotes",
    "T1271s4-D1": "low eukaryotes",
    "T1271s5-D1": "low eukaryotes",
    "T1271s5-D2": "low eukaryotes",
    "T1271s6-D1": "low eukaryotes",
    "T1271s7-D1": "low eukaryotes",
    "T1271s8-D1": "low eukaryotes",
    "T1271s8-D2": "low eukaryotes",
    "T1272s2-D1": "high eukaryotes",
    "T1272s6-D1": "high eukaryotes",
    "T1272s8-D1": "high eukaryotes",
    "T1272s9-D1": "high eukaryotes",
    "T1274-D1": "prokaryotes",
    "T1276-D1": "high eukaryotes",
    "T1278-D1": "prokaryotes",
    "T1279-D1": "prokaryotes",
    "T1279-D2": "prokaryotes",
    "T1280-D1": "viral",
    "T1284-D1": "viral",
    "T1292-D1": "prokaryotes",
    "T1294-D1": "low eukaryotes",
    "T1295-D1": "nanobody",
    "T1295-D2": "nanobody",
    "T1295-D3": "prokaryotes",
    "T1298-D1": "prokaryotes",
    "T1298-D2": "prokaryotes",
    "T1299-D1": "archaea",
}

# print all keys without repeating
keys = set()
for key, value in species_information.items():
    keys.add(value)
print(keys)

species_plot_dict = {'prokaryotes':[], 'eukaryotes':[], 'viral':[]}

for k, v in GDT_HA_percentile_dict.items():
    species = species_information[k]
    if species == 'prokaryotes' or species == 'archaea':
        species_plot_dict['prokaryotes'].append(v)
    elif species == 'high eukaryotes' or species == 'low eukaryotes':
        species_plot_dict['eukaryotes'].append(v)
    elif species == 'viral':
        species_plot_dict['viral'].append(v)
    else:
        print(f'{k} not in species_plot_dict, will skip')
        continue


# plot boxplot using species_plot_dict
plt.figure(figsize=(3, 6), dpi=300)
sns.boxplot(data=[species_plot_dict['prokaryotes'], species_plot_dict['eukaryotes'], species_plot_dict['viral']],
            # patch_artist=True,
            width=0.6,
            notch=True,
            showmeans=True,
            boxprops=dict(edgecolor='blue', facecolor='lightblue'),
            medianprops=dict(color='black'),
            meanprops={"marker":"o", "markerfacecolor":"black", "markeredgecolor":"red"},
            flierprops=dict(marker='o', markerfacecolor='white',  markeredgecolor='black'))

sns.stripplot(data=[species_plot_dict['prokaryotes'], species_plot_dict['eukaryotes'], species_plot_dict['viral']],
              color='black',
              alpha=0.7,
              jitter=True)

# calculate p-values
_, p12 = stats.ttest_ind(species_plot_dict['prokaryotes'], species_plot_dict['eukaryotes'], equal_var=False)
_, p23 = stats.ttest_ind(species_plot_dict['eukaryotes'], species_plot_dict['viral'], equal_var=False)
_, p13 = stats.ttest_ind(species_plot_dict['prokaryotes'], species_plot_dict['viral'], equal_var=False)

# Add p-values with brackets
def add_bracket(x1, x2, y, p):
    plt.plot([x1, x1, x2, x2], [y, y+1, y+1, y], 'k-')
    plt.text((x1+x2)/2, y+1, f'p={p:.3f}', ha='center', va='bottom', fontsize=10)

max_val = max([max(species_plot_dict['prokaryotes']), max(species_plot_dict['eukaryotes']), max(species_plot_dict['viral'])])
add_bracket(0, 1, max_val + 2, p12)
# add_bracket(1, 2, max_val + 4, p23)
# add_bracket(0, 2, max_val + 6, p13)


plt.xticks([0, 1, 2], ['prokaryotes', 'eukaryotes', 'viruses'], fontsize=20, rotation=60, ha='right')
plt.grid(axis='y', linestyle='--', linewidth=0.5, alpha=0.7)
plt.ylabel('GDT-HA 95% percentile', fontsize=20)
plt.yticks(fontsize=20)
# plt.title('GDT-HA 95% percentile for different species', fontsize=20)
plt.show()

In [ ]:
target_source_dict = {
    "T1201-D1": "homo",
    "T1206-D1": "homo",
    "T1207-D1": "mono",
    "T1208s1-D1": "hetero",
    "T1208s2-D1": "hetero",
    "T1210-D1": "mono",
    "T1212-D1": "hybrid",
    "T1218-D1": "homo",
    "T1218-D2": "homo",
    "T1218-D3": "homo",
    "T1220s1-D1": "hetero",
    "T1226-D1": "mono",
    "T1227s1-D1": "hetero",
    "T1228-D1": "hybrid",
    "T1228-D2": "hybrid",
    "T1228-D3": "hybrid",
    "T1228-D4": "hybrid",
    "T1230s1-D1": "hetero",
    "T1231-D1": "mono",
    "T1234-D1": "homo",
    "T1235-D1": "homo",
    "T1237-D1": "homo",
    "T1239-D1": "hybrid",
    "T1239-D2": "hybrid",
    "T1239-D3": "hybrid",
    "T1239-D4": "hybrid",
    "T1240-D1": "homo",
    "T1240-D2": "homo",
    "T1243-D1": "mono",
    "T1244s1-D1": "hetero",
    "T1245s1-D1": "hetero",
    "T1245s2-D1": "hetero",
    "T1246-D1": "mono",
    "T1249-D1": "homo",
    "T1249v1-D1": "homo",
    # there is a problem in the nomenclature of the target, it should be T1249v1-D1, but T1249-D1 is used in other places. we will keep both for now
    "T1257-D1": "homo",
    "T1259-D1": "homo",
    "T1266-D1": "mono",
    "T1267s1-D1": "hetero",
    "T1267s1-D2": "hetero",
    "T1267s2-D1": "hetero",
    "T1269-D1": "homo",
    "T1269-D2": "homo",
    "T1269-D3": "homo",
    "T1270-D1": "homo",
    "T1270-D2": "homo",
    "T1271s1-D1": "hybrid",
    "T1271s2-D1": "hybrid",
    "T1271s3-D1": "hybrid",
    "T1271s4-D1": "hybrid",
    "T1271s5-D1": "hybrid",
    "T1271s5-D2": "hybrid",
    "T1271s6-D1": "hybrid",
    "T1271s7-D1": "hybrid",
    "T1271s8-D1": "hybrid",
    "T1271s8-D2": "hybrid",
    "T1272s2-D1": "hetero",
    "T1272s6-D1": "hetero",
    "T1272s8-D1": "hetero",
    "T1272s9-D1": "hetero",
    "T1274-D1": "mono",
    "T1276-D1": "hybrid",
    "T1278-D1": "mono",
    "T1279-D1": "mono",
    "T1279-D2": "mono",
    "T1280-D1": "mono",
    "T1284-D1": "mono",
    "T1292-D1": "homo",
    "T1294-D1": "homo",
    "T1295-D1": "homo",
    "T1295-D2": "homo",
    "T1295-D3": "homo",
    "T1298-D1": "homo",
    "T1298-D2": "homo",
    "T1299-D1": "mono",
}

# print all keys without repeating
keys = set()
for key, value in target_source_dict.items():
    keys.add(value)
print(keys)

target_source_plot_dict = {'monomer':[], 'homoligomer':[], 'heteroligomer':[], 'hybrid':[]}

for k, v in GDT_HA_percentile_dict.items():
    source = target_source_dict[k]
    if source == 'mono':
        target_source_plot_dict['monomer'].append(v)
    elif source == 'homo':
        target_source_plot_dict['homoligomer'].append(v)
    elif source == 'hetero':
        target_source_plot_dict['heteroligomer'].append(v)
    elif source == 'hybrid':
        target_source_plot_dict['hybrid'].append(v)
    else:
        print(f'{k} not in target_source_plot_dict, will skip')
        continue

# plot boxplot using target_source_plot_dict
plt.figure(figsize=(4, 6), dpi=300)
sns.boxplot(data=[target_source_plot_dict['monomer'], target_source_plot_dict['homoligomer'], target_source_plot_dict['heteroligomer'], target_source_plot_dict['hybrid']],
            # patch_artist=True,
            width=0.6,
            notch=True,
            showmeans=True,
            boxprops=dict(edgecolor='blue', facecolor='lightblue'),
            medianprops=dict(color='black'),
            meanprops={"marker":"o", "markerfacecolor":"black", "markeredgecolor":"red"},
            flierprops=dict(marker='o', markerfacecolor='white',  markeredgecolor='black'))

sns.stripplot(data=[target_source_plot_dict['monomer'], target_source_plot_dict['homoligomer'], target_source_plot_dict['heteroligomer'], target_source_plot_dict['hybrid']],
              color='black', alpha=0.7, jitter=True)

_, p12 = stats.ttest_ind(target_source_plot_dict['monomer'], target_source_plot_dict['homoligomer'], equal_var=False)
_, p13 = stats.ttest_ind(target_source_plot_dict['monomer'], target_source_plot_dict['heteroligomer'], equal_var=False)
_, p14 = stats.ttest_ind(target_source_plot_dict['monomer'], target_source_plot_dict['hybrid'], equal_var=False)
_, p23 = stats.ttest_ind(target_source_plot_dict['homoligomer'], target_source_plot_dict['heteroligomer'], equal_var=False)
_, p24 = stats.ttest_ind(target_source_plot_dict['homoligomer'], target_source_plot_dict['hybrid'], equal_var=False)
_, p34 = stats.ttest_ind(target_source_plot_dict['heteroligomer'], target_source_plot_dict['hybrid'], equal_var=False)

def add_bracket(x1, x2, y, p):
    plt.plot([x1, x1, x2, x2], [y, y+1, y+1, y], 'k-')
    plt.text((x1+x2)/2, y+1, f'p={p:.3f}', ha='center', va='bottom', fontsize=10)

max_val = max([max(target_source_plot_dict['monomer']), max(target_source_plot_dict['homoligomer']), max(target_source_plot_dict['heteroligomer']), max(target_source_plot_dict['hybrid'])])
# add_bracket(0, 1, max_val + 2, p12)
# add_bracket(0, 2, max_val + 4, p13)
# add_bracket(0, 3, max_val + 6, p14)
# add_bracket(1, 2, max_val + 8, p23)
# add_bracket(1, 3, max_val + 10, p24)
# add_bracket(2, 3, max_val + 12, p34)
# add_bracket(1, 2, max_val + 2, p23)

plt.xticks([0, 1, 2, 3], ['monomer', 'homoligomer', 'heteroligomer', 'hybrid'], fontsize=20, rotation=60, ha='right')
plt.ylabel('GDT-HA 95% percentile', fontsize=20)
plt.yticks(fontsize=20)
plt.grid(axis='y', linestyle='--', linewidth=0.5, alpha=0.7)
# plt.title('GDT-HA 95% percentile for different target sources', fontsize=20)
plt.show()

In [ ]:

target_experiment_dict = {
    "T1201-D1": "X-ray",
    "T1206-D1": "X-ray",
    "T1207-D1": "X-ray",
    "T1208s1-D1": "unknown",
    "T1208s2-D1": "unknown",
    "T1210-D1": "cryo-EM",
    "T1212-D1": "unknown",
    "T1218-D1": "X-ray",
    "T1218-D2": "X-ray",
    "T1218-D3": "X-ray",
    "T1220s1-D1":"unknown",
    "T1226-D1": "NMR",
    "T1227s1-D1":"unknown",
    "T1228-D1": "unknown",
    "T1228-D2": "unknown",
    "T1228-D3": "unknown",
    "T1228-D4": "unknown",
    "T1230s1-D1":"unknown",
    "T1231-D1": "X-ray",
    "T1234-D1": "cryo-EM",
    "T1235-D1": "cryo-EM",
    "T1237-D1": "X-ray",
    "T1239-D1": "unknown",
    "T1239-D2": "unknown",
    "T1239-D3": "unknown",
    "T1239-D4": "unknown",
    "T1240-D1": "X-ray",
    "T1240-D2": "X-ray",
    "T1243-D1": "X-ray",
    "T1244s1-D1":"unknown",
    "T1245s1-D1":"unknown",
    "T1245s2-D1":"unknown",
    "T1246-D1": "X-ray",
    "T1249-D1": "cryo-EM",
    "T1249v1-D1": "cryo-EM",
    # there is a problem in the nomenclature of the target, it should be T1249v1-D1, but T1249-D1 is used in other places. we will keep both for now
    "T1257-D1": "cryo-EM",
    "T1259-D1": "X-ray",
    "T1266-D1": "X-ray",
    "T1267s1-D1": "unknown",
    "T1267s1-D2": "unknown",
    "T1267s2-D1": "unknown",
    "T1269-D1": "cryo-EM",
    "T1269-D2": "cryo-EM",
    "T1269-D3": "cryo-EM",
    "T1270-D1": "cryo-EM",
    "T1270-D2": "cryo-EM",
    "T1271s1-D1":"unknown",
    "T1271s2-D1":"unknown",
    "T1271s3-D1":"unknown",
    "T1271s4-D1":"unknown",
    "T1271s5-D1":"unknown",
    "T1271s5-D2":"unknown",
    "T1271s6-D1":"unknown",
    "T1271s7-D1":"unknown",
    "T1271s8-D1":"unknown",
    "T1271s8-D2":"unknown",
    "T1272s2-D1":"unknown",
    "T1272s6-D1":"unknown",
    "T1272s8-D1":"unknown",
    "T1272s9-D1":"unknown",
    "T1274-D1": "X-ray",
    "T1276-D1": "unknown",
    "T1278-D1": "X-ray",
    "T1279-D1": "X-ray",
    "T1279-D2": "X-ray",
    "T1280-D1": "X-ray",
    "T1284-D1": "X-ray",
    "T1292-D1": "X-ray",
    "T1294-D1": "X-ray",
    "T1295-D1": "cryo-EM",
    "T1295-D2": "cryo-EM",
    "T1295-D3": "cryo-EM",
    "T1298-D1": "X-ray",
    "T1298-D2": "X-ray",
    "T1299-D1": "X-ray",
}

target_experiment_plot_dict = {'X-ray':[], 'cryo-EM':[], 'NMR':[], 'unknown':[]}

for k, v in GDT_HA_percentile_dict.items():
    experiment = target_experiment_dict[k]
    if experiment == 'X-ray':
        target_experiment_plot_dict['X-ray'].append(v)
    elif experiment == 'cryo-EM':
        target_experiment_plot_dict['cryo-EM'].append(v)
    elif experiment == 'NMR':
        target_experiment_plot_dict['NMR'].append(v)
    elif experiment == 'unknown':
        target_experiment_plot_dict['unknown'].append(v)
    else:
        print(f'{k} not in target_experiment_plot_dict, will skip')
        continue

plt.figure(figsize=(4, 6), dpi=300)
sns.boxplot(data=[target_experiment_plot_dict['X-ray'], target_experiment_plot_dict['cryo-EM'], target_experiment_plot_dict['NMR'], target_experiment_plot_dict['unknown']],
            # patch_artist=True,
            width=0.6,
            notch=True,
            showmeans=True,
            boxprops=dict(edgecolor='blue', facecolor='lightblue'),
            medianprops=dict(color='black'),
            meanprops={"marker":"o", "markerfacecolor":"black", "markeredgecolor":"red"},
            flierprops=dict(marker='o', markerfacecolor='white',  markeredgecolor='black'))

sns.stripplot(data=[target_experiment_plot_dict['X-ray'], target_experiment_plot_dict['cryo-EM'], target_experiment_plot_dict['NMR'], target_experiment_plot_dict['unknown']],
              color='black', alpha=0.7, jitter=True)

_, p12 = stats.ttest_ind(target_experiment_plot_dict['X-ray'], target_experiment_plot_dict['cryo-EM'], equal_var=False)
_, p13 = stats.ttest_ind(target_experiment_plot_dict['X-ray'], target_experiment_plot_dict['NMR'], equal_var=False)
_, p14 = stats.ttest_ind(target_experiment_plot_dict['X-ray'], target_experiment_plot_dict['unknown'], equal_var=False)
_, p23 = stats.ttest_ind(target_experiment_plot_dict['cryo-EM'], target_experiment_plot_dict['NMR'], equal_var=False)
_, p24 = stats.ttest_ind(target_experiment_plot_dict['cryo-EM'], target_experiment_plot_dict['unknown'], equal_var=False)
_, p34 = stats.ttest_ind(target_experiment_plot_dict['NMR'], target_experiment_plot_dict['unknown'], equal_var=False)

def add_bracket(x1, x2, y, p):
    plt.plot([x1, x1, x2, x2], [y, y+1, y+1, y], 'k-')
    plt.text((x1+x2)/2, y+1, f'p={p:.3f}', ha='center', va='bottom', fontsize=10)

max_val = max([max(target_experiment_plot_dict['X-ray']), max(target_experiment_plot_dict['cryo-EM']), max(target_experiment_plot_dict['NMR']), max(target_experiment_plot_dict['unknown'])])
add_bracket(0, 1, max_val + 2, p12)
# add_bracket(0, 2, max_val + 4, p13)
# add_bracket(0, 3, max_val + 6, p14)
# add_bracket(1, 2, max_val + 8, p23)
# add_bracket(1, 3, max_val + 10, p24)
# add_bracket(2, 3, max_val + 12, p34)

plt.xticks([0, 1, 2, 3], ['X-ray', 'cryo-EM', 'NMR', 'unknown'], fontsize=20, rotation=60, ha='right')
plt.ylabel('GDT-HA 95% percentile', fontsize=20)
plt.yticks(fontsize=20)
plt.grid(axis='y', linestyle='--', linewidth=0.5, alpha=0.7)
# plt.title('GDT-HA 95% percentile for different target experiments', fontsize=20)
plt.show()

In [ ]:
# T1271s8-D1	69.85	0.5
# T1234-D1	89.85	0.57
# T1239-D1	77.59	0.63
# T1295-D3	75.32	0.64
# T1295-D1	80.94	0.65
# T1235-D1	94.11	0.66
# T1271s2-D1	71.65	0.72
# T1295-D2	87.32	0.72
# T1271s5-D1	72.61	0.85
# T1239-D3	70.3	0.88
# T1271s6-D1	87.63	0.88
# T1239-D4	61.63	0.94
# T1271s4-D1	68.79	0.95
# T1228-D1	78.34	0.98
# T1271s3-D1	73.45	0.98
# T1228-D4	57.58	0.99
# T1259-D1	92.03	1
# T1271s8-D2	65.37	1.01
# T1218-D1	75.88	1.03
# T1276-D1	84.66	1.03
# T1292-D1	97.45	1.04
# T1271s5-D2	73.36	1.06
# T1280-D1	92.2	1.06
# T1271s1-D1	53.85	1.15
# T1218-D2	78	1.16
# T1239-D2	88.52	1.17
# T1243-D1	76.13	1.19
# T1228-D3	53.53	1.2
# T1244s1-D1	93.53	1.21
# T1270-D2	71.83	1.32
# T1267s2-D1	72.56	1.33
# T1267s1-D2	81.69	1.35
# T1231-D1	85.85	1.39
# T1249-D1	73.43	1.39
# T1271s7-D1	80.32	1.39
# T1272s8-D1	78.56	1.39
# T1272s9-D1	71.72	1.4
# T1272s6-D1	80.97	1.43
# T1278-D1	97.18	1.43
# T1208s1-D1	83.49	1.45
# T1208s2-D1	91.58	1.48
# T1206-D1	94.03	1.5
# T1272s2-D1	75.77	1.51
# T1284-D1	77.27	1.51
# T1270-D1	81.54	1.52
# T1245s1-D1	85.14	1.54
# T1299-D1	84.52	1.58
# T1228-D2	87.22	1.61
# T1245s2-D1	79.87	1.63
# T1218-D3	86.81	1.64
# T1212-D1	69.3	1.65
# T1201-D1	77.13	1.7
# T1227s1-D1	77.61	1.7
# T1267s1-D1	58.6	1.73
# T1237-D1	79	1.84
# T1294-D1	90.34	1.84
# T1274-D1	97.8	1.86
# T1210-D1	48.55	1.88
# T1279-D2	72.69	1.94
# T1269-D2	68.23	1.97
# T1230s1-D1	74.75	2.01
# T1298-D1	77.52	2.03
# T1279-D1	96.67	2.05
# T1257-D1	68.08	2.12
# T1269-D3	69.06	2.12
# T1269-D1	71.29	2.17
# T1207-D1	50.2	2.72
# T1240-D1	94.71	2.72
# T1298-D2	77.03	2.78
# T1246-D1	87.65	2.82
# T1240-D2	82.38	2.99
# T1220s1-D1	45.69	3.37
# T1266-D1	71.67	3.42
# T1226-D1	48.26	3.57
target_molp_score_dict = {
    "T1271s8-D1": 0.5,
    "T1234-D1": 0.57,
    "T1239-D1": 0.63,
    "T1295-D3": 0.64,
    "T1295-D1": 0.65,
    "T1235-D1": 0.66,
    "T1271s2-D1": 0.72,
    "T1295-D2": 0.72,
    "T1271s5-D1": 0.85,
    "T1239-D3": 0.88,
    "T1271s6-D1": 0.88,
    "T1239-D4": 0.94,
    "T1271s4-D1": 0.95,
    "T1228-D1": 0.98,
    "T1271s3-D1": 0.98,
    "T1228-D4": 0.99,
    "T1259-D1": 1.0,
    "T1271s8-D2": 1.01,
    "T1218-D1": 1.03,
    "T1276-D1": 1.03,
    "T1292-D1": 1.04,
    "T1271s5-D2": 1.06,
    "T1280-D1": 1.06,
    "T1271s1-D1": 1.15,
    "T1218-D2": 1.16,
    "T1239-D2": 1.17,
    "T1243-D1": 1.19,
    "T1228-D3": 1.2,
    "T1244s1-D1": 1.21,
    "T1270-D2": 1.32,
    "T1267s2-D1": 1.33,
    "T1267s1-D2": 1.35,
    "T1231-D1": 1.39,
    "T1249-D1": 1.39,
    "T1249v1-D1": 1.39,
    # there is a problem in the nomenclature of the target, it should be T1249v1-D1, but T1249-D1 is used in other places. we will keep both for now
    "T1271s7-D1": 1.39,
    "T1272s8-D1": 1.39,
    "T1272s9-D1": 1.4,
    "T1272s6-D1": 1.43,
    "T1278-D1": 1.43,
    "T1208s1-D1": 1.45,
    "T1208s2-D1": 1.48,
    "T1206-D1": 1.5,
    "T1272s2-D1": 1.51,
    "T1284-D1": 1.51,
    "T1270-D1": 1.52,
    "T1245s1-D1": 1.54,
    "T1299-D1": 1.58,
    "T1228-D2": 1.61,
    "T1245s2-D1": 1.63,
    "T1218-D3": 1.64,
    "T1212-D1": 1.65,
    "T1201-D1": 1.7,
    "T1227s1-D1": 1.7,
    "T1267s1-D1": 1.73,
    "T1237-D1": 1.84,
    "T1294-D1": 1.84,
    "T1274-D1": 1.86,
    "T1210-D1": 1.88,
    "T1279-D2": 1.94,
    "T1269-D2": 1.97,
    "T1230s1-D1": 2.01,
    "T1298-D1": 2.03,
    "T1279-D1": 2.05,
    "T1257-D1": 2.12,
    "T1269-D3": 2.12,
    "T1269-D1": 2.17,
    "T1207-D1": 2.72,
    "T1240-D1": 2.72,
    "T1298-D2": 2.78,
    "T1246-D1": 2.82,
    "T1240-D2": 2.99,
    "T1220s1-D1": 3.37,
    "T1266-D1": 3.42,
    "T1226-D1": 3.57
}

# if molp is less than 1.7, it is a good target
# if molp is greater than 1.7, it is a bad target
target_molp_dict = {}

for key, value in target_molp_score_dict.items():
    if value < 1.7:
        target_molp_dict[key] = 'good'
    else:
        target_molp_dict[key] = 'bad'


target_molp_plot_dict = {'good':[], 'bad':[]}

for k, v in GDT_HA_percentile_dict.items():
    if target_molp_dict[k] == 'good':
        target_molp_plot_dict['good'].append(v)
    elif target_molp_dict[k] == 'bad':
        target_molp_plot_dict['bad'].append(v)
    else:
        print(f'{k} not in target_molp_dict, will skip')
        continue

plt.figure(figsize=(2.5, 6), dpi=300)
sns.boxplot(data=[target_molp_plot_dict['good'], target_molp_plot_dict['bad']],
            width=0.6,
            notch=True,
            showmeans=True,
            boxprops=dict(edgecolor='blue', facecolor='lightblue'),
            medianprops=dict(color='black'),
            meanprops={"marker":"o", "markerfacecolor":"black", "markeredgecolor":"red"},
            flierprops=dict(marker='o', markerfacecolor='white',  markeredgecolor='black'))

sns.stripplot(data=[target_molp_plot_dict['good'], target_molp_plot_dict['bad']],
              color='black', alpha=0.7, jitter=True)

_, p12 = stats.ttest_ind(target_molp_plot_dict['good'], target_molp_plot_dict['bad'], equal_var=False)

def add_bracket(x1, x2, y, p):
    plt.plot([x1, x1, x2, x2], [y, y+1, y+1, y], 'k-')
    plt.text((x1+x2)/2, y+1, f'p={p:.3f}', ha='center', va='bottom', fontsize=10)

max_val = max([max(target_molp_plot_dict['good']), max(target_molp_plot_dict['bad'])])
# add_bracket(0, 1, max_val + 2, p12)

plt.xticks([0, 1], ['good targets mol-probity', 'bad targets mol-probity'], fontsize=12, rotation=60, ha='right')
plt.ylabel('GDT-HA 95% percentile', fontsize=20)
plt.yticks(fontsize=20)
plt.grid(axis='y', linestyle='--', linewidth=0.5, alpha=0.7)
plt.show()


In [ ]:
# T1220s1-D1	45.69	0.0121419
# T1284-D1	77.27	0.0168067
# T1245s2-D1	79.87	0.0172414
# T1234-D1	89.85	0.0347216
# T1231-D1	85.85	0.0435543
# T1271s1-D1	53.85	0.0530372
# T1257-D1	68.08	0.0597967
# T1271s8-D2	65.37	0.0653595
# T1271s4-D1	68.79	0.143638
# T1245s1-D1	85.14	0.145098
# T1235-D1	94.11	0.150943
# T1210-D1	48.55	0.156631
# T1271s8-D1	69.85	0.176577
# T1249-D1	73.43	0.185934
# T1206-D1	94.03	0.190492
# T1271s3-D1	73.45	0.210012
# T1271s2-D1	71.65	0.213398
# T1271s7-D1	80.32	0.380147
# T1218-D2	78	0.384345
# T1226-D1	48.26	0.429678
# T1266-D1	71.67	0.442276
# T1271s6-D1	87.63	0.457982
# T1271s5-D2	73.36	0.465597
# T1208s2-D1	91.58	0.505143
# T1298-D2	77.03	0.528777
# T1271s5-D1	72.61	0.614167
# T1218-D1	75.88	0.66789
# T1212-D1	69.3	0.817045
# T1272s9-D1	71.72	0.880797
# T1230s1-D1	74.75	0.898779
# T1276-D1	84.66	1.17452
# T1239-D3	70.3	1.31567
# T1208s1-D1	83.49	1.36723
# T1272s2-D1	75.77	1.48613
# T1237-D1	79	1.84649
# T1201-D1	77.13	1.98127
# T1218-D3	86.81	2.01661
# T1228-D3	53.53	2.20479
# T1239-D1	77.59	2.23958
# T1228-D1	78.34	2.28278
# T1272s8-D1	78.56	2.29637
# T1267s1-D2	81.69	2.80819
# T1267s2-D1	72.56	2.81292
# T1299-D1	84.52	2.85243
# T1207-D1	50.2	3.4103
# T1272s6-D1	80.97	4.16729
# T1228-D4	57.58	4.98013
# T1239-D4	61.63	4.99528
# T1274-D1	97.8	5.32283
# T1227s1-D1	77.61	5.87798
# T1269-D3	69.06	6.84722
# T1240-D2	82.38	6.85793
# T1267s1-D1	58.6	7.10887
# T1269-D2	68.23	7.70233
# T1269-D1	71.29	8.91136
# T1298-D1	77.52	11.8385
# T1292-D1	97.45	12.9713
# T1240-D1	94.71	13.1976
# T1259-D1	92.03	13.5124
# T1228-D2	87.22	14.8239
# T1239-D2	88.52	14.8655
# T1246-D1	87.65	16.5954
# T1278-D1	97.18	23.7562
# T1243-D1	76.13	27.4885
# T1280-D1	92.2	30.5439
# T1294-D1	90.34	36.034
# T1270-D1	81.54	58.5832
# T1270-D2	71.83	59.8667
# T1244s1-D1	93.53	71.6316
target_neff_score_dict = {
    'T1220s1-D1': 0.0121419,
    'T1284-D1': 0.0168067,
    'T1245s2-D1': 0.0172414,
    'T1234-D1': 0.0347216,
    'T1231-D1': 0.0435543,
    'T1271s1-D1': 0.0530372,
    'T1257-D1': 0.0597967,
    'T1271s8-D2': 0.0653595,
    'T1271s4-D1': 0.143638,
    'T1245s1-D1': 0.145098,
    'T1235-D1': 0.150943,
    'T1210-D1': 0.156631,
    'T1271s8-D1': 0.176577,
    'T1249-D1': 0.185934,
    'T1249v1-D1': 0.185934,
    # there is a problem in the nomenclature of the target, it should be T1249v1-D1, but T1249-D1 is used in other places. we will keep both for now
    'T1206-D1': 0.190492,
    'T1271s3-D1': 0.210012,
    'T1271s2-D1': 0.213398,
    'T1271s7-D1': 0.380147,
    'T1218-D2': 0.384345,
    'T1226-D1': 0.429678,
    'T1266-D1': 0.442276,
    'T1271s6-D1': 0.457982,
    'T1271s5-D2': 0.465597,
    'T1208s2-D1': 0.505143,
    'T1298-D2': 0.528777,
    'T1271s5-D1': 0.614167,
    'T1218-D1': 0.66789,
    'T1212-D1': 0.817045,
    'T1272s9-D1': 0.880797,
    'T1230s1-D1': 0.898779,
    'T1276-D1': 1.17452,
    'T1239-D3': 1.31567,
    'T1208s1-D1': 1.36723,
    'T1272s2-D1': 1.48613,
    'T1237-D1': 1.84649,
    'T1201-D1': 1.98127,
    'T1218-D3': 2.01661,
    'T1228-D3': 2.20479,
    'T1239-D1': 2.23958,
    'T1228-D1': 2.28278,
    'T1272s8-D1': 2.29637,
    'T1267s1-D2': 2.80819,
    'T1267s2-D1': 2.81292,
    'T1299-D1': 2.85243,
    'T1207-D1': 3.4103,
    'T1272s6-D1': 4.16729,
    'T1228-D4': 4.98013,
    'T1239-D4': 4.99528,
    'T1274-D1': 5.32283,
    'T1227s1-D1': 5.87798,
    'T1269-D3': 6.84722,
    'T1240-D2': 6.85793,
    'T1267s1-D1': 7.10887,
    'T1269-D2': 7.70233,
    'T1269-D1': 8.91136,
    'T1298-D1': 11.8385,
    'T1292-D1': 12.9713,
    'T1240-D1': 13.1976,
    'T1259-D1': 13.5124,
    'T1228-D2': 14.8239,
    'T1239-D2': 14.8655,
    'T1246-D1': 16.5954,
    'T1278-D1': 23.7562,
    'T1243-D1': 27.4885,
    'T1280-D1': 30.5439,
    'T1294-D1': 36.034,
    'T1270-D1': 58.5832,
    'T1270-D2': 59.8667,
    'T1244s1-D1': 71.6316
}


# if neff is less than 0.5, it has low Neff
# if neff is greater than 0.5 and less than 5, it has medium Neff
# if neff is greater than 5, it has high Neff

target_neff_dict = {}

for key, value in target_neff_score_dict.items():
    if value < 0.5:
        target_neff_dict[key] = 'low'
    elif value < 5:
        target_neff_dict[key] = 'medium'
    else:
        target_neff_dict[key] = 'high'

target_neff_plot_dict = {'low':[], 'medium':[], 'high':[]}

for k, v in GDT_HA_percentile_dict.items():
    try:
        if target_neff_dict[k] == 'low':
            target_neff_plot_dict['low'].append(v)
        elif target_neff_dict[k] == 'medium':
            target_neff_plot_dict['medium'].append(v)
        elif target_neff_dict[k] == 'high':
            target_neff_plot_dict['high'].append(v)
    except KeyError:
        print(f'{k} not in target_neff_dict, will skip')
        continue
plt.figure(figsize=(4, 6), dpi=300)
sns.boxplot(data=[target_neff_plot_dict['low'], target_neff_plot_dict['medium'], target_neff_plot_dict['high']],
            width=0.6,
            notch=True,
            showmeans=True,
            boxprops=dict(edgecolor='blue', facecolor='lightblue'),
            medianprops=dict(color='black'),
            meanprops={"marker":"o", "markerfacecolor":"black", "markeredgecolor":"red"},
            flierprops=dict(marker='o', markerfacecolor='white',  markeredgecolor='black'))

sns.stripplot(data=[target_neff_plot_dict['low'], target_neff_plot_dict['medium'], target_neff_plot_dict['high']],
              color='black', alpha=0.7, jitter=True)

_, p12 = stats.ttest_ind(target_neff_plot_dict['low'], target_neff_plot_dict['medium'], equal_var=False)
_, p23 = stats.ttest_ind(target_neff_plot_dict['medium'], target_neff_plot_dict['high'], equal_var=False)
_, p13 = stats.ttest_ind(target_neff_plot_dict['low'], target_neff_plot_dict['high'], equal_var=False)

def add_bracket(x1, x2, y, p):
    plt.plot([x1, x1, x2, x2], [y, y+1, y+1, y], 'k-')
    plt.text((x1+x2)/2, y+1, f'p={p:.3f}', ha='center', va='bottom', fontsize=10)

max_val = max([max(target_neff_plot_dict['low']), max(target_neff_plot_dict['medium']), max(target_neff_plot_dict['high'])])
add_bracket(0, 1, max_val + 2, p12)
add_bracket(1, 2, max_val + 4, p23)
add_bracket(0, 2, max_val + 6, p13)

plt.xticks([0, 1, 2], ['low Neff (<0.5)', 'medium Neff (0.5-5)', 'high Neff (>5)'], fontsize=16, rotation=60, ha='right')
plt.ylabel('Neff', fontsize=20)
plt.yticks(fontsize=20)
plt.grid(axis='y', linestyle='--', linewidth=0.5, alpha=0.7)
plt.show()


In [ ]:
# test code

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats

# 示例数据
data = {
    'Group': ['AF2 and AF3'] * 30 + ['AF2 only'] * 30 + ['AF3 only'] * 30 + ['neither'] * 30,
    'Value': np.random.normal(70, 10, 30).tolist() + np.random.normal(65, 10, 30).tolist() +
             np.random.normal(60, 10, 30).tolist() + np.random.normal(55, 10, 30).tolist()
}

df = pd.DataFrame(data)

# 创建图表
plt.figure(figsize=(8, 6))
sns.boxplot(x='Group', y='Value', data=df, notch=True,  showmeans=True,
            meanprops={"marker":"o", "markerfacecolor":"red", "markeredgecolor":"black"})

# 添加散点图
sns.stripplot(x='Group', y='Value', data=df, color='black', alpha=0.5, jitter=True)

# 添加显著性标记
p_value = 0.003
plt.title(f'P={p_value}', fontsize=16)

# 设置标签
plt.xlabel('')
plt.ylabel('average GDTHA for each target', fontsize=12)
plt.xticks(rotation=45)

# 显示图表
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Create sample data
np.random.seed(42)
group1 = np.random.normal(70, 10, 30)
group2 = np.random.normal(65, 10, 30)
group3 = np.random.normal(60, 10, 30)

# Calculate p-values
_, p12 = stats.ttest_ind(group1, group2)
_, p23 = stats.ttest_ind(group2, group3)
_, p13 = stats.ttest_ind(group1, group3)

# Create boxplot
plt.figure(figsize=(6, 8))
sns.boxplot(data=[group1, group2, group3],
            color='lightblue',
            boxprops={'edgecolor': 'blue'},
            notch=True,
            showmeans=True)

# Add p-values with brackets
def add_bracket(x1, x2, y, p):
    plt.plot([x1, x1, x2, x2], [y, y+1, y+1, y], 'k-')
    plt.text((x1+x2)/2, y+1, f'p={p:.3f}', ha='center', va='bottom')

max_val = max([max(group1), max(group2), max(group3)])
add_bracket(0, 1, max_val + 2, p12)
add_bracket(1, 2, max_val + 4, p23)
add_bracket(0, 2, max_val + 6, p13)

plt.xticks([0, 1, 2], ['Group 1', 'Group 2', 'Group 3'])
plt.ylabel('Value')
plt.show()